## Modifying The Environment

Great! Now lets have our newly-trained ant run to a few different points around the area and see how it behaves.

In [ ]:
import IPython.display as ipd
ipd.Image("../animations/stuck_ant.png")

Oh. As it turns out, the way that the ant is set up, it learns to run off into the positive-x direction.. and ONLY that.  It was never given any opportunity to have any other experiences to learn from, and so it didn't.  The result of the ant running off to the right is very effective at that one, specific, thing - but completely fragile to any other request, or changes to the environment it might run in.

So, lets look at how we might go about modifying our ant and its environment to make the agent a bit more flexible.
Of course, you can define the entirety of your environment, its actors, the steps and rewards, etc, by yourself from scratch, but that's a lot of work to be done if you don't need to.  Instead, we rely on the wonderful python `subclass` to override certain things about the ant and its environments we might want to alter, while leaving everything else intact.  If you're new to the concept of a subclass, it's rather straight-forward: define `class MyNewClass(AnOldClass):` in your code and `MyNewClass` will carry forward all the functions that `AnOldClass` had, except where you overwrite them with a function having the same name. In our repository, if you now look at [override_ant.py](https://github.com/dillonroach/TD3/blob/master/override_ant.py) we take all the pieces from [the original ant](https://github.com/bulletphysics/bullet3/tree/master/examples/pybullet/gym/pybullet_envs) and give new names to things, while also exposing key functions in each piece we might want to modify.  The subclassing here gets messy, so here's a quick rundown of what inherits from what:

```
 class MyAntBulletEnv(WalkerBaseBulletEnv): # <- this is our entry point to the new environment
       self.robot = MyAnt()
                      \\\
                   class MyAnt(MyWalkerBase): # MyAntBulletEnv sets the robot as MyAnt
                                \\\
                                class MyWalkerBase(WalkerBase): #MyAnt subclasses MyWalkerBase, itself from WalkerBase
                                
                                    def init():
                                        ...
                                    def step():
                                        ...
                   
```

If you take a look at `MyWalkerBase()` and compare to `WalkerBase()` from [the original](https://github.com/bulletphysics/bullet3/blob/master/examples/pybullet/gym/pybullet_envs/robot_locomotors.py) we've copied over `init()` and `step()` as they were written, then tweaked small things.  In this first go, we've only used all this to set the robot target x and y to a new location: 20m, 20m.

In order to now use this new, modified environment, we simply add some registration boilerplate to our notebook:

```
from gym.envs.registration import registry, make, spec


def register(id, *args, **kvargs):
  if id in registry.env_specs:
    return
  else:
    return gym.envs.registration.register(id, *args, **kvargs)
    
register(id='MyAntBulletEnv-v0',
         entry_point='override_ant:MyAntBulletEnv',
         max_episode_steps=1000,
         reward_threshold=2500.0)
```
And then set `"env" : "MyAntBulletEnv-v0",` instead of the original, in the arguments dictionary.  You can see an example of this change in [TD3notebook-MyAnt.ipynb](https://github.com/dillonroach/TD3/blob/master/TD3notebook-MyAnt.ipynb).  If you run this notebook, you'll end up with an ant that runs to (20,20) instead of off to the right for a kilometer.

In [ ]:
ipd.Image("../animations/diagon_anty.png")